This notebook is yet another attempt at multivariate statitics. We'll start by importing everything and the kitchen sink, in case we need it. 

In [27]:
import numpy as np
import skbio
import scipy

import pandas as pd
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
% matplotlib inline


import americangut.ag_dictionary as agdic
import americangut.diversity_analysis as agdiv
import americangut.notebook_environment as agenv
from americangut.ag_data import AgData

Next, let's select the data set and rarefaction depth we wish to use.

In [13]:
bodysite = 'fecal'
sequence_trim = '100nt'
rarefaction_depth = '10k'

use_subset = False
use_one_sample = True

Next, we'll load the data, and remove outliers.

In [3]:
data = AgData(bodysite=bodysite, 
                    trim=sequence_trim, 
                    depth=rarefaction_depth, 
                    sub_participants=use_subset, 
                    one_sample=use_one_sample)

data.drop_alpha_outliers()
data.drop_bmi_outliers()
data.clean_age()

In [4]:
ibd = {u'10317.000012951', u'10317.000017746', u'10317.000001620',
       u'10317.000001748', u'10317.000009382', u'10317.000001544',
       u'10317.000005953', u'10317.000009004', u'10317.000013010',
       u'10317.000014579', u'10317.000001827', u'10317.000013023',
       u'10317.000009157', u'10317.000001362', u'10317.000001135',
       u'10317.000004855', u'10317.000005878', u'10317.000007719',
       u'10317.000009126', u'10317.000007068', u'10317.000010134',
       u'10317.000010112', u'10317.000003300', u'10317.000002830',
       u'10317.000014236', u'10317.000012936', u'10317.000013575',
       u'10317.000001047', u'10317.000005360', u'10317.000001333',
       u'10317.000015581', u'10317.000003189', u'10317.000001128',
       u'10317.000015907', u'10317.000017338', u'10317.000013062',
       u'10317.000013595', u'10317.000009533', u'10317.000010546',
       u'10317.000001685', u'10317.000004163', u'10317.000001751',
       u'10317.000013551', u'10317.000005889', u'10317.000002036',
       u'10317.000002783', u'10317.000004157', u'10317.000001364',
       u'10317.000001622', u'10317.000004783', u'10317.000008999',
       u'10317.000009149', u'10317.000001895', u'10317.000011375',
       u'10317.000014880', u'10317.000001291', u'10317.000010876',
       u'10317.000014607', u'10317.000014987', u'10317.000002271',
       u'10317.000003898', u'10317.000009144', u'10317.000002482',
       u'10317.000014608', u'10317.000013134', u'10317.000001647',
       u'10317.000011959', u'10317.000009626', u'10317.000004025',
       u'10317.000014291', u'10317.000001351', u'10317.000002859',
       u'10317.000014118', u'10317.000004612', u'10317.000018383',
       u'10317.000001575', u'10317.000004161', u'10317.000015873',
       u'10317.000005971', u'10317.000004192', u'10317.000001322',
       u'10317.000004162', u'10317.000002336', u'10317.000010147',
       u'10317.000006673', u'10317.000004752', u'10317.000005851',
       u'10317.000009236', u'10317.000001363', u'10317.000014458',
       u'10317.000011093', u'10317.000009164', u'10317.000005810',
       u'10317.000003047', u'10317.000015849', u'10317.000004790',
       }
actual = set(ibd).intersection(data.map_.index)

Finally, let's pick our response varaible.

In [5]:
metric = 'PD_whole_tree'
response = '%s_%s' % (metric, rarefaction_depth)

Next, we're going to loop through our dataset and clean up the question responses and number them.

In [6]:
for group_name in agdic.dictionary.iterkeys():
    if group_name == 'AGE_CORRECTED':
        continue
    question = agdic.ag_dictionary(group_name)
    if group_name == 'COUNTRY':
        question.order = ['USA', 'Australia', 'Canada', 'United Kingdom']
    elif group_name == 'IBD':
        question.order = ['No', 'Yes']
    data.clean_group(question)
    question.label_order(data.map_)

We'll also update the IBD data to account for both participants who provided a diagnosis, rather than a diagnosis manner.

In [7]:
data.map_.loc[actual, 'IBD'] = '(1) Yes'

Let's also add a column for the natural log of age, since this may be a better shape for the relationship with alpha diversity.

In [9]:
data.map_['lnAGE'] = data.map_['AGE_CORRECTED'].apply(lambda x: np.log(x))

In [148]:
vars1 = [('AGE_CORRECTED', 'lnAGE'), 'SEX', 'COUNTRY', 'IBD', 'CHICKENPOX',
         'RACE', 'ANTIBIOTIC_HISTORY', 'TYPES_OF_PLANTS', 'ALCOHOL_FREQUENCY', #'IBS',
         # 'BOWEL_MOVEMENT_FREQUENCY', 'BOWEL_MOVEMENT_QUALITY',
         'CSECTION', 'DIABETES', 'DIET_TYPE',
         'DRINKING_WATER_SOURCE', 'EXERCISE_FREQUENCY', 'EXERCISE_LOCATION',
         'FLOSSING_FREQUENCY', 'MIGRAINE', 'RACE', 'SEX', 'SLEEP_DURATION', 
         'SMOKING_FREQUENCY','TYPES_OF_PLANTS',
        ]
vars2 = [('AGE_CORRECTED', 'lnAGE'), 'SEX', 'COUNTRY', 'IBD', 'CHICKENPOX',
         'ALCOHOL_FREQUENCY', 'ANTIBIOTIC_HISTORY', 'IBS',
         'BOWEL_MOVEMENT_FREQUENCY', 'BOWEL_MOVEMENT_QUALITY',
         'COUNTRY', 'CSECTION', 'DIABETES', 'DIET_TYPE',
         'DRINKING_WATER_SOURCE', 'EXERCISE_FREQUENCY', 'EXERCISE_LOCATION',
         'FLOSSING_FREQUENCY', 'MIGRAINE', 'RACE', 'SEX', 'SLEEP_DURATION', 
         'SMOKING_FREQUENCY', # 'TYPES_OF_PLANTS',
        ]

In [149]:
def _equation_builder(vars_, last_eq=None, response=None):
    """Builds a set of equations trying multiple predictor options"""
    # Checks enough variables are defined
    if last_eq is None and response is None:
        raise ValueError('A response or last equation must be specified.')

    # Checks the class of vars
    if isinstance(vars_, str):
        vars_ = [vars_]

    # Builds the equation
    if last_eq is None:
        return ['%s ~ %s' % (response, pred) for pred in vars_]
    else:
        eqs = []
        for pred in vars_:
            if '&' in pred:
                eqs.append(last_eq.replace(' + %s' % pred[1:], ''))
            else:
                eqs.append('%s + %s' % (last_eq, pred))

    return eqs

In [150]:
def _populate_fit_check(fits, var_, id_=1, dname=None):
    """Updates the fit_check information"""
    fit_check = []
    if isinstance(var_, str):
        var_ = [var_]

    for idy, fit_ in enumerate(fits):
        n = fit_.nobs
        k = fit_.df_model
        aicc = fit_.aic + (2 * k * (k + 1) / (n - k - 1))
        d_score = scipy.stats.kstest(fit_.resid.values, 'norm')[0]

        fit_check.append(pd.Series({'data_set': dname,
                                    'equation':  fit_.model.formula,
                                    'var': var_[idy],
                                    'n': n,
                                    'k': k,
                                    'aic': fit_.aic,
                                    'aicc': aicc,
                                    'D_score': d_score,
                                    'pearson_r2': fit_.rsquared,
                                    'adj_r2': fit_.rsquared_adj,
                                    'cond no': fit_.condition_number},
                                   name=id_ + idy))
    return pd.DataFrame(fit_check)

In [151]:
def _check_watch(model_watch, prev_id):
    """..."""
    if model_watch is None:
        model_watch = pd.DataFrame(data=np.zeros((0, 11)),
                                   columns=['data_set', 'equation', 'var', 'n',
                                            'k', 'aic', 'aicc', 'D_score',
                                            'pearson_r2', 'adj_r2', 'cond no'])
        id_ = 1
        prev_id = None
        prev_eq = None
    else:
        id_ = max(model_watch.index) + 1
        if prev_id is None:
            models = pd.DataFrame(model_watch).transpose()
            ranked = models.sort(['aicc'], inplace=False).index
            prev_id = ranked[0]
        prev_eq = model_watch.loc[prev_id, 'equation']
    return model_watch, id_, prev_id, prev_eq

In [152]:
def _identify_best_model(fit_check, prev_id):
    """..."""
    fit_check['ref'] = prev_id
    prev_r2 = fit_check.loc[prev_id, 'adj_r2']
    prev_aicc = fit_check.loc[prev_id, 'aicc']
    fit_check['score'] = -10000*(((fit_check.adj_r2 - prev_r2) *
                                 (fit_check.aicc - prev_aicc)) /
                                 fit_check['cond no'])
    prev_id = fit_check.loc[fit_check.score == fit_check.score.max()].index[0]
    return prev_id

In [153]:
def olf_build_model(response, predictors, data, dname=None, model_watch=None,
                    prev_id=None):
    """Builds up a series of Ordinary Least Squares Models

    """
    # Looks for the last model being referenced as the best
    model_watch, id_, prev_id, prev_eq = _check_watch(model_watch, prev_id)

    # Builds up the model
    for var_ in predictors:
        # Builds up the list of equations to be added
        eqs = _equation_builder(var_, prev_eq, response)
        num_var = len(eqs)

        # Fits the equations
        fits = [smf.ols(eq, data=data).fit() for eq in eqs]

        # Summarizes the fit
        fit_check = _populate_fit_check(fits, var_, id_, dname)

        # Updates the fit check with the best previous model
        check_ids = fit_check.index.values
        if prev_id is None:
            prev_id = min(fit_check.index.values)
        else:
            fit_check.loc[prev_id] = model_watch.loc[prev_id]

        # Identifies the best model
        prev_id = _identify_best_model(fit_check, prev_id)

        model_watch = pd.concat((model_watch, fit_check.loc[check_ids]))

        # Gets the best fit equation
        prev_eq = model_watch.loc[prev_id, 'equation']

        # Advances the counter
        id_ = id_ + num_var

    return model_watch, prev_id


In [154]:
model_watch, prev_id = olf_build_model(
        response=response,
        predictors=vars1, 
        data=data.map_.loc[data.map_[vars1[1:]].dropna().index],
        dname='all_data',
        model_watch=None,
        prev_id=None
        )

In [155]:
sort_ = model_watch.sort_values('score', ascending=False)

In [156]:
sort_.loc[sort_.index[0], 'equation']

'PD_whole_tree_10k ~ lnAGE + COUNTRY + IBD + CHICKENPOX + RACE + ANTIBIOTIC_HISTORY'

In [157]:
print smf.ols(sort_.loc[sort_.index[0], 'equation'],
        data.map_.loc[data.map_[vars1[1:]].dropna().index]
       ).fit().summary()

                            OLS Regression Results                            
Dep. Variable:      PD_whole_tree_10k   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     13.27
Date:                Mon, 08 Feb 2016   Prob (F-statistic):           2.79e-24
Time:                        14:39:38   Log-Likelihood:                -4136.7
No. Observations:                1321   AIC:                             8297.
Df Residuals:                    1309   BIC:                             8360.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------------